In [ ]:
from metrics.evaluation_metrics import *
from utils.util_torch import *

In [ ]:
from new.configs import *
from new.utils import * 
from new.models import *

In [ ]:
opt = parse_config()
print(opt)

In [ ]:
opt.batch_size = batch_size
opt.swap_axis = True
if len(opt.checkpoint_path) == 0:
    opt.checkpoint_path = None 
opt.device = "cuda:%s" % opt.cuda if opt.cuda!="" else "cpu"
opt.shuffle = not opt.warm_start

In [ ]:
train_data = PointCloudDataSet(opt)
data_collator = PointCloudDataCollator(opt)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=opt.batch_size, drop_last=opt.drop_last, 
shuffle=opt.shuffle, collate_fn = data_collator, num_workers=torch.cuda.device_count() * 4)

In [ ]:
train_data[0].unsqueeze(0).shape

In [ ]:
#from utils.util_torch import visualize

In [ ]:
#visualize(train_data[0].unsqueeze(0))

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
point_clouds = next(iter(train_loader))

In [ ]:
mean_point_cloud = torch.mean(point_clouds, dim = 0)

In [ ]:
mean_point_cloud = mean_point_cloud.expand([16,-1,-1])

In [ ]:
mean_point_cloud.shape

In [ ]:
num_cols,num_rows = 4, 4
idx = None

In [ ]:
def show_point_clouds(point_clouds):
    if point_clouds.shape[1] < 10: 
        point_clouds = np.swapaxes(point_clouds, 1, 2)
    num_clouds = len(point_clouds)
    # num_rows = min(num_rows, num_clouds // num_cols + 1)

    fig = plt.figure(figsize=(num_cols * 4, num_rows * 4))
    for i, pts in enumerate(point_clouds[:num_cols*num_rows]):
        #print(i)
        if point_clouds.shape[2] == 3: 
            ax = plt.subplot(num_rows, num_cols, i+1, projection='3d')
            plt.subplots_adjust(0,0,1,1,0,0)
            #ax.axis('off')
            if idx is not None:
                ax.set_title(str(idx[i]))
            ax.scatter(pts[:,0], pts[:,2], pts[:,1], marker='.', s=50, c=pts[:,2], cmap=plt.get_cmap('gist_rainbow'))
        else: 
            ax = plt.subplot(num_rows, num_cols, i+1)
            plt.subplots_adjust(0,0,1,1,0,0)
            # ax.axis('off')
            if idx is not None:
                ax.set_title(str(idx[i]))
            ax.scatter(pts[:,1], -pts[:,0], marker='.', s=30)
    
    plt.show()

In [ ]:
show_point_clouds(point_clouds)

In [ ]:
show_point_clouds(mean_point_cloud)

In [ ]:
plt.show()

In [ ]:
point_clouds.shape

# TEST

In [ ]:
from new.configs import *
from new.utils import * 
from new.models import *

In [ ]:
opt = parse_config()

In [ ]:
opt.batch_size = batch_size
opt.swap_axis = True
if len(opt.checkpoint_path) == 0:
    opt.checkpoint_path = None 
opt.device = "cuda:%s" % opt.cuda if opt.cuda!="" else "cpu"
opt.shuffle = not opt.warm_start

In [ ]:
net = NetWrapper()

In [ ]:
net.load_state_dict(torch.load("runs/Oct23_11-51-39_yizhou-Z370-AORUS-Gaming-5/model.pth"))

In [ ]:
net.eval()

In [ ]:
opt.category

In [ ]:
ref_pcs = np.load("data/%s_test.npy" % opt.category)

In [ ]:
ref_pcs.shape

In [ ]:
net = net.cuda()

In [ ]:
syn_pcs = net.sample_x(n=2)

In [ ]:
syn_pcs.shape

In [ ]:
show_point_clouds(syn_pcs)

In [ ]:
from utils.util_torch import *

In [ ]:
res = quantitative_analysis(syn_pcs.data.numpy(), ref_pcs, 16, full=False)

In [ ]:
res

In [ ]:
num_cols,num_rows = 4, 4
idx = None
if point_clouds.shape[1] < 10: 
    point_clouds = np.swapaxes(point_clouds, 1, 2)
num_clouds = len(point_clouds)
# num_rows = min(num_rows, num_clouds // num_cols + 1)

fig = plt.figure(figsize=(num_cols * 4, num_rows * 4))
for i, pts in enumerate(point_clouds[:num_cols*num_rows]):
    #print(i)
    if point_clouds.shape[2] == 3: 
        ax = plt.subplot(num_rows, num_cols, i+1, projection='3d')
        plt.subplots_adjust(0,0,1,1,0,0)
        #ax.axis('off')
        if idx is not None:
            ax.set_title(str(idx[i]))
        ax.scatter(pts[:,0], pts[:,2], pts[:,1], marker='.', s=50, c=pts[:,2], cmap=plt.get_cmap('gist_rainbow'))
    else: 
        ax = plt.subplot(num_rows, num_cols, i+1)
        plt.subplots_adjust(0,0,1,1,0,0)
        # ax.axis('off')
        if idx is not None:
            ax.set_title(str(idx[i]))
        ax.scatter(pts[:,1], -pts[:,0], marker='.', s=30)

In [ ]:
ax